In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter

In [2]:
data=pd.read_csv("UCoursera_Courses.csv")
data.drop(["Unnamed: 0"],axis=1,inplace=True)

In [3]:
def remove(myString):
  myString = re.sub(r"[\n\t]*", "", myString)
  myString = re.sub(r'[.,"\'-?:!;]', '', myString)
  return myString

nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')
def clean_text(txt):
    txt="".join([c for c in txt if c not in string.punctuation])
    tokens=re.split('\W+',txt)
    txt=[word for word in tokens if word not in stopwords]
    return txt

nltk.download('wordnet')
wn=nltk.WordNetLemmatizer()
def lemm(token_txt):
    text=[wn.lemmatize(word) for word in token_txt]
    return text

[nltk_data] Downloading package stopwords to C:\Users\Vanshika
[nltk_data]     Nehra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Vanshika
[nltk_data]     Nehra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
data['course_title_l']= data['course_title'].apply(lambda x: remove(x))
data['course_organization_l']= data['course_organization'].apply(lambda x: remove(x))
data['course_title_l']=data['course_title_l'].apply(lambda x:clean_text(x))
data['course_organization_l']= data['course_organization_l'].apply(lambda x: clean_text(x))
data['course_title_l']=data['course_title_l'].apply(lambda x: lemm(x))
data['course_organization_l']=data['course_organization_l'].apply(lambda x: lemm(x))

In [5]:
data['course_Certificate_type']=data['course_Certificate_type'].mask(data['course_Certificate_type']=='PROFESSIONAL CERTIFICATE','PROFESSIONAL_CERTIFICATE')

In [6]:
def create_soup(x):
    return ' '.join(x['course_title_l']) + ' ' + ' '.join(x['course_organization_l']) + ' ' + x['course_Certificate_type'] + ' ' + x['course_difficulty']

data['soup'] = data.apply(create_soup, axis=1)

In [7]:
def get_course():
    course = input("What Course are you interested in?")
    courses = " ".join(["".join(n.split()) for n in course.lower().split(',')])
    return course

def get_searchTerms():
    searchTerms = "" 
    courses = get_course()
    if courses != 'skip':
        searchTerms = courses
    return searchTerms

In [8]:
def make_recommendation(data=data):
    searchTerms = get_searchTerms()
    searchTerms = remove(searchTerms)
    searchTerms = clean_text(searchTerms)
    searchTerms = lemm(searchTerms)
    new_row = data.iloc[-1,:].copy()
    new_row.iloc[-1] = " ".join(searchTerms)
    data = data.append(new_row)
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(data['soup'])  
    cosine_sim2 = cosine_similarity(count_matrix, count_matrix) #getting a similarity matrix
    sim_scores = list(enumerate(cosine_sim2[-1,:]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    ranked_titles = []
    for i in range(1, 4):
        indx = sim_scores[i][0]
        if (sim_scores[i][1] != 0):
            ranked_titles.append([data['course_title'].iloc[indx], data['course_rating'].iloc[indx], data['course_students_enrolled'].iloc[indx]])
#     ranked_titles= sorted(ranked_titles, key=itemgetter(1), reverse=True)
    if len(ranked_titles)==0:
        return -1
    else:
        return ranked_titles

In [ ]:
make_recommendation()

In [11]:
print("Hey There!")

L=[]
while(True):
    print()
    L = make_recommendation()
    if L !=-1:
        for i in range(len(L)):
            print(L[i])
    choice = input("Do you want to see more type of courses? (Yes/No)")
    if ((choice.lower() == "yes") or (choice.lower() == "y")):
        continue;
    else:
        print("Hope you have your courses. See you Again!")
        break;

Hey There!

What Course are you interested in?Chinese
['Chinese for HSK 1', 4.8, '22k']
['Learn Chinese: HSK Test Preparation', 4.8, '43k']
['Learn Mandarin Chinese', 4.8, '39k']
Do you want to see more type of courses? (Yes/No)y

What Course are you interested in?Cloud Computing
['Cloud Computing Basics (Cloud 101)', 4.5, '37k']
['Introduction to Cloud Computing', 4.6, '2.6k']
['Cloud Computing', 4.4, '110k']
Do you want to see more type of courses? (Yes/No)y

What Course are you interested in?finance
['Supply Chain Finance and Blockchain Technology', 4.6, '4.9k']
['Behavioral Finance', 4.4, '55k']
['Essentials of Corporate Finance', 4.6, '54k']
Do you want to see more type of courses? (Yes/No)n
Hope you have your courses. See you Again!
